In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
!unzip /content/drive/MyDrive/char-rnn-keras-master.zip

Archive:  /content/drive/MyDrive/char-rnn-keras-master.zip
   creating: char-rnn-keras-master/
  inflating: char-rnn-keras-master/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/char-rnn-keras-master/
  inflating: __MACOSX/char-rnn-keras-master/._.DS_Store  
  inflating: char-rnn-keras-master/.gitignore  
   creating: char-rnn-keras-master/__pycache__/
  inflating: char-rnn-keras-master/__pycache__/model.cpython-36.pyc  
   creating: char-rnn-keras-master/data/
  inflating: char-rnn-keras-master/data/char_to_idx.json  
  inflating: char-rnn-keras-master/data/input.txt  
   creating: char-rnn-keras-master/logs/
  inflating: char-rnn-keras-master/logs/training_log.csv  
   creating: char-rnn-keras-master/model/
  inflating: char-rnn-keras-master/model/model.10.h5  
  inflating: char-rnn-keras-master/model/model.100.h5  
  inflating: char-rnn-keras-master/model/weights.10.h5  
  inflating: char-rnn-keras-master/model/weights.100.h5  
  inflating: char-rnn-keras-master/model/weigh

In [4]:
file = open("/content/char-rnn-keras-master/data/input.txt")
for i in file:
  print(i)
file.close()

Streaming output truncated to the last 5000 lines.
"G"g2G BAG|"G"ded dBG|"C"cec "G"BdB|"D7"ABA "G"G2:|

P:B

d|"D7"def "G"gfg|"D7"aba afd|"D7"dfg aga|"G"bgg g2d|

"D"def "C"gfe|"G"ded dBG|"C"cec "G"BdB|"D7"ABA "G"G2|





X: 171

T:The Lassie's Fancy

% Nottingham Music Database

S:FTB 2/39, via EF

Y:AB

M:6/8

K:D

P:A

f|"D"d2A AFA|"D"B2A AFA|"D"d2A AGF|"Em"EFE "A7"ABc|

"D"d2A AFA|"G"B2A "A"ABc|"Bm"dgf "A7"eac|"D"d3 d2:|

P:B

f/2g/2|"D"afd dfa|"G"gec "A7"ceg|"D"fed dcd|"A7"ecA A2f/2g/2|

"D"afd dfa|"G"gec "A7"ceg|"D"fad "A7"eac|"D"d3 "A7"d2f/2g/2|

"D"afd dfa|"G"gec "A7"ceg|"D"fed dcd|"A7"ecA A2G|

"D"FAd FAd|"G"GBd GBd|"D"Agf "A7"eac|"D"d3 d2||





X: 172

T:St. Lawrence Jig

% Nottingham Music Database

S:Trad, arr Phil Rowe

M:6/8

K:D

A|"D"ABA FAd|f2d A2d|"A"c2B G2B|"D"B2A F3|

ABA FAd|"D"f2d A2d|"A"c2e ABc|"D"d3 -d2::

f|"G"a2g B3|"D"g2f A3|"A"d2c cBc|"D"edB "D7"A3|"G"a2g B3|"D"g2f A3|"A7"d2c cBc\

|"D"d3 -d2:|





X: 173

T:Leaping Jack

% Nottingham Music Database

S:Joh

In [5]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding
import os

def build_model(batch_size, seq_len, vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 512, batch_input_shape=(batch_size, seq_len)))
    for i in range(3):
        model.add(LSTM(256, return_sequences=True, stateful=True))
        model.add(Dropout(0.2))

    model.add(TimeDistributed(Dense(vocab_size))) 
    model.add(Activation('softmax'))
    return model

dir="Music_model"

def save_weights(epoch,model):
  if not os.path.exists(dir):
    
    os.makedirs(dir)
  model.save_weights(os.path.join(dir, 'weights{}.h5'.format(epoch)))


def create_batch(T,vocab_size):

  length=T.shape[0]
  batch_chars=int(length/BATCH_SIZE)

  for i in range(0,batch_chars-SEQ_LENGTH,SEQ_LENGTH):
    x=np.zeros((BATCH_SIZE,SEQ_LENGTH))
    y=np.zeros((BATCH_SIZE, SEQ_LENGTH, vocab_size))

    for j in range(0,BATCH_SIZE):
      for k in range(0,SEQ_LENGTH):
        x[j,k]=T[batch_chars*j+i+k]
        y[j,k,T[batch_chars*j+i+k+1]]=1
    
    yield x,y


def Train(text_path,epochs,save_frq):
 
 with open(text_path,"r") as f:
    chars=set()
    for i in f:
      chars.update(set(i))

    #vocab size
    vocab_size=len(chars)
  
    #encoding vocab
    chars_idx={j:i for (i,j) in enumerate(sorted(list(chars)))}
    idx_chars={j:i for (i,j) in chars_idx.items()}

    print(f.read())
    
    #converting text to numerical representation
    f.seek(0)
    T=np.array([chars_idx[i] for c in f for i in c])

    #model_architecture
    model = build_model(BATCH_SIZE, SEQ_LENGTH, vocab_size)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    for epoch in range(1,epochs+1):
      losses=[]
      accs=[]
      for f,(x,y) in enumerate(create_batch(T,vocab_size)):
        loss,acc=model.train_on_batch(x,y)
        losses.append(loss)
        accs.append(accs)
       # print("Batch-{} ---- loss = {},acc = {}".format(f,loss,acc))
      print("epoch {} ----- loss = {},acc = {}".format(epoch,np.mean(loss),np.mean(acc)))
      if (epoch)% save_frq==0:

        save_weights(epoch,model)
        print('Saved checkpoint to', 'weights.{}.h5'.format(epoch))



In [32]:
tf.keras.backend.clear_session()
BATCH_SIZE=16
SEQ_LENGTH=64
Train("/content/char-rnn-keras-master/data/input.txt",100,10)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (16, 64, 512)             44032     
                                                                 
 lstm (LSTM)                 (16, 64, 256)             787456    
                                                                 
 dropout (Dropout)           (16, 64, 256)             0         
                                                                 
 lstm_1 (LSTM)               (16, 64, 256)             525312    
                                                                 
 dropout_1 (Dropout)         (16, 64, 256)             0         
                                                                 
 lstm_2 (LSTM)               (16, 64, 256)             525312    
                                                                 
 dropout_2 (Dropout)         (16, 64, 256)             

In [52]:
from numpy.random.bit_generator import SeedSequence
def build_model(batch_size, seq_len, vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 512, batch_input_shape=(batch_size, seq_len)))
    for i in range(3):
        model.add(LSTM(256, return_sequences=(i!=2), stateful=True))
        model.add(Dropout(0.2))

    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    return model


def Generate_music(model_weights_path,out_seq,seed,text_path):

 with open(text_path,"r") as f:
    chars=set()
    for i in f:
      chars.update(set(i))

    #vocab size
    vocab_size=len(chars)
      
    #encoding vocab
    chars_idx={j:i for (i,j) in enumerate(sorted(list(chars)))}
    idx_chars={j:i for (i,j) in chars_idx.items()}
    
 model=build_model(1,1,vocab_size)
 model.load_weights(model_weights_path)
 generated=[seed]
 for i in range(out_seq):

    output=model.predict_on_batch(np.array([[generated[-1]]])).ravel()
    sample=np.random.choice(range(vocab_size),p=output)
    generated.append(sample)
 return ''.join(idx_chars[c] for c in generated)

In [54]:
out=Generate_music("/content/char-rnn-keras-master/model/model.100.h5",1024,53,"/content/char-rnn-keras-master/data/input.txt")
print(out)

[2"A"[F/2E/2A/2c/2d/2B/2/2 B/2c/2A/2||


X: 233
T:The Pinection
% Nottingham Music Database
S:EF
Y:AB
M:4/4
K:D
M:6/8
P:A
f/2g/2|"D"f2A dff|"D"a2a b2g|"D"fgf fed|"Em"efg "A7"efg|
"D"faf dcd|"G"BGB "A7"c2g|"D"agf "A7"edc|"D"dAF D2||


X: 201
T:Sot of the Cork
% Nottingham Music Database
S:Kevin Briggs, via EF
Y:AB
M:6/8
K:D
P:A
A|"D"d2d fdd|"A"ecA cde|"G"dcB A2G|"A7"ABA "D"D2:|
P:B
|:c/2d/2|"G"g2e dcB|"D7"A2A ABc|"G"d2B "D7"cBA|"G"G2G "D7"AFA|"G"G3 G2:|
K:Bb
P:C
|:d|"G"g2d dBd|"C"e2e "Am"ede|"D7"fed cBA|"G"G3 -G2:|
P:B
d|"G"g2g gdB|"D7"ded cBA|"G"GAG "C7"GGE|"F"A3 -A2:|


X: 205
T:Silver Lucky Lins
% Nottingham Music Database
Y:AAB
S:Kevin Briggs, via EF
M:6/8
K:D
P:A
|:A|"D"DFD DFA|"D"ded "A/c+"cBA|"Bm"dfg "A7"ecA|"D"d3 -d2:|


X: 230
T:Paddy the Wander
% Nottingham Music Database
S:AA, via EF
Y:AB
M:6/8
K:G
P:A
d/2d/2|"G"dBB B3|"C"cec "G"BAG|"D7"AGA cBA|"G"BGG G2:|
P:B
g/2a/2|"G"bag dgb|"G"bge dBG|"C"cec "G"BdB|"D7"Add "G"G2:|
P:B
"C"ece "G"ddB|"C"cde "G/b"dBd|"Am"ecA "G"BAG|"Am"A2A A